In [2]:
import os
vipshome = r'C:\Users\Kevin\Downloads\vips-dev-8.14\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips

In [3]:
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import skimage.measure
import scipy.stats as stats
import time
import shutil
from tqdm import tqdm
from skimage.measure import label, regionprops_table

In [ ]:
### Find tissue composition of all the label/classes defined by the mask that was created by using the MATLAB deeplabv3 segmentation model:

# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3]
desired_df_name1 = all_df[all_df.scan_batch == 4]
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
desired_names = [x for x in desired_names if x[-3:] == "_he"]
desired_names = [x+".png" for x in desired_names]

In [ ]:
# save xlsx for individual masks:
masksrc = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined'
desired_mask_src = [os.path.join(masksrc,x) for x in desired_names]
desired_mask_names = [os.listdir(x) for x in desired_mask_src]
save_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\composition'
class_list = list(['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','bloodvessel','ecm','fat','white'])
comps_df = pd.DataFrame(index = class_list)

for idx in tqdm(range(0,len(desired_mask_src)),total = len(desired_mask_src),desc="Image processed",colour='red'):
    print("processing image: {}".format(desired_mask_src[idx]))
    fn, _ = os.path.splitext(desired_mask_names[idx])
    # check if xlsx exists in destination
    save_src_xlsx = os.path.join(save_src, fn + ".xlsx")
    if os.path.exists(save_src_xlsx):
        print("xlsx for image {} already exists, skipping to next".format(desired_mask_src[idx]))
        continue
    DLarr = Image.open(desired_mask_src[idx])
    DLarr = np.array(DLarr)
    DLarr = DLarr[DLarr < 12] #background 12 shouldn't count in the composition
    DLarr_flatten = DLarr.flatten()
    hist = np.bincount(DLarr_flatten) #dl mask
    hist = hist/np.sum(hist)*100
    count_list = list(hist)
    del(count_list[0])
    count_list = [np.round(x,3) for x in count_list]
    ratio_class_dict = dict(zip(class_list,count_list))
    tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
    comps_df = pd.concat([comps_df,pd.DataFrame.from_dict(tmp_df)],axis=1)
    comps_df.dropna()
    comps_df.rename(columns = {0:'composition'}, inplace=True)
    comps_df.to_excel(save_src_xlsx)

In [81]:
# src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_11932_2_he.png'
# class_list = list(['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','bloodvessel','ecm','fat','white'])
# comps_df = pd.DataFrame(index = class_list)
# DLarr = Image.open(src)
# DLarr = np.array(DLarr)
# DLarr = DLarr[DLarr < 12] #background 12 shouldn't count in the composition
# DLarr_flatten = DLarr.flatten()
# hist = np.bincount(DLarr_flatten) #dl mask
# hist = hist/np.sum(hist)*100
# count_list = list(hist)
# del(count_list[0])
# count_list = [np.round(x,3) for x in count_list]
# ratio_class_dict = dict(zip(class_list,count_list))
# tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
# comps_df = pd.concat([comps_df,pd.DataFrame.from_dict(tmp_df)],axis=1)
# comps_df.dropna()
# comps_df.rename(columns = {0:'composition'}, inplace=True)

In [82]:
comps_df

,composition
corneum,1.641
spinosum,8.653
hairshaft,0.426
hairfollicle,1.084
smoothmuscle,4.234
oil,0.038
sweat,0.036
nerve,2.214
bloodvessel,2.044
ecm,78.321
